In [ ]:
!pip install transformers
!pip install cltk 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.0/845.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:

In [ ]:
import numpy as np
import pandas as pd
import nltk 
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import ast
from numpy.linalg import norm

In [ ]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True )

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Final_Project/final.csv')
df['Lemmatized Text'] = df['Lemmatized Text'].apply(lambda x: ast.literal_eval(x))

array(['Ovid', 'Horace', 'Virgil', 'Catullus', 'Seneca', 'Tacitus',
       'Juvenal', 'Martial', 'Tertullian', 'St.Augustine', 'Ausonius'],
      dtype=object)

#  Opening df and Organizing 

In [ ]:
gol = df.loc[df['Period']=='Golden']
sil = df.loc[df['Period']=='Silver']
pat = df.loc[df['Period']=='Patristic']

In [ ]:
## create label and sentence list
sentences = df['Joined Lemmas'].values

#check distribution of data based on labels
print("Distribution of data based on labels: ",df['Period'].value_counts())

Distribution of data based on labels:  Silver       24936
Golden       14975
Patristic     6949
Name: Period, dtype: int64


In [ ]:
#training data
gol_train =[]
for x in gol['Lemmatized Text']:
  gol_train.append(x)

sil_train =[]
for x in sil['Lemmatized Text']:
  sil_train.append(x)

pat_train =[]
for x in pat['Lemmatized Text']:
  pat_train.append(x)

#Secondary Cleaning 

In [ ]:
stopwords = ['ab', 'ac', 'ad', 'adhic', 'aliqui', 'aliquis', 'an', 'ante', 'apud', 'at', 'atque','aut', 'autem', 'cum', 'cur', 'de', 'deinde', 'dum', 'ego', 'enim', 'ergo', 
'es','ego','est', 'et', 'etiam', 'etsi', 'ex', 'fio', 'haud', 'hic', 'iam', 'idem', 'igitur', 'ille', 'in', 'infra', 'inter', 'interim', 'ipse', 'is', 'ita', 
'magis', 'modo', 'mox', 'nam', 'ne', 'nec', 'necque', 'neque', 'nisi', 'non', 'nos', 'o', 'ob', 'per', 'possum', 'post', 'pro', 'quae', 'quam', 
'quare', 'qui', 'quia', 'quicumque', 'quidem', 'quilibet', 'quis', 'quisnam', 'quisquam', 'quisque', 'quisquis', 'quo','quod', 'quoniam', 'sed', 'si',
'sic', 'sive', 'sub', 'sui', 'super','suus', 'tam', 'tamen', 'trans', 'tu', 'tum', 'ubi', 'ut', 'uel', 'uero', 'vobis', 'tibi','mihi',
'.', '?', '!','quasi']


In [ ]:
gold_train = []
for x in gol_train: 
  for y in x: 
    if y in stopwords: 
      x.remove(y)
  gold_train.append(x)

patr_train = []
for x in pat_train: 
  for y in x: 
    if y in stopwords: 
      x.remove(y)
  patr_train.append(x)

In [ ]:
silver_train = []
for x in sil_train: 
  for y in x: 
    if y in stopwords: 
      x.remove(y)
  silver_train.append(x)

# Word2Vec

In [ ]:
gold_voc = set(sum(gold_train,[]))
silver_voc = set(sum(silver_train,[]))
pat_voc = set(sum(patr_train,[]))

In [ ]:
vector_size = 300
window = 2

gol_model = Word2Vec(sentences=gold_train, vector_size=vector_size, window=window, min_count=1)
sil_model = Word2Vec(sentences=silver_train, vector_size=vector_size, window=window, min_count=1)
pat_model = Word2Vec(sentences=patr_train, vector_size=vector_size, window=window, min_count=1)

In [ ]:
vector = sil_model.wv['animus']  # get numpy vector of a word
sims = sil_model.wv.most_similar('animus', topn=10)  # get other similar words

In [ ]:
def j_sim(word1,word2):
  wordlist1 = (list(zip(*word1))[0])
  wordlist2 = (list(zip(*word2))[0])
  intersection = set(wordlist1).intersection(set(wordlist2))
  union = set(wordlist1).union(set(wordlist2))
  sim = len(intersection) / len(union)
  return sim #,len(intersection),len(union)


In [ ]:
w1_gold = gol_model.wv.most_similar('colla', topn=10)
print(list(zip(*w1_gold))[0])

('auris', 'aqua', 'litor', 'lego', 'flamma', 'equus', 'ager', 'caput', 'suus', 'ingens')


In [ ]:
w1_silver = sil_model.wv.most_similar('colla', topn=10)
print(list(zip(*w1_silver))[0])

('acies', 'flumen', 'simul', 'mons', 'pes', 'dens', 'niger', 'iaceo', 'cohortus', 'altus')


In [ ]:
j_sim(w1_gold,w1_silver)

0.0

In [ ]:
w1 = gol_model.wv.most_similar('os', topn=50)
w2 = sil_model.wv.most_similar('os', topn=50)

j_sim(w1,w2)

0.19047619047619047

# Comparasions 

## gold to silver

In [ ]:
#gold to silver 

gs_info = []

for x in gold_voc: 
  if x in silver_voc: 
    w1 = gol_model.wv.most_similar(x, topn=50)
    w2 = sil_model.wv.most_similar(x, topn=50)
    sim = j_sim(w1,w2)
    topgold = list(zip(*w1))[0] 
    topsilver = list(zip(*w2))[0]
    intersect = set(topgold).intersection(set(topsilver))
    gs_info.append((x,sim,intersect,topgold,topsilver))
    


In [ ]:
data = list(sorted(gs_info))
df = pd.DataFrame(data, columns =['Word','Jaccard Sim',"Shared Set",'Gold Set','Silver Set'])
df.head(1)

,Word,Jaccard Sim,Shared Set,Gold Set,Silver Set
0,1,0.0,{},"(prope, tempus, paruus, mens, homo, cupio, fel...","(obscure, murex, vocantur, belgicus, cunnum, m..."


In [ ]:
df.sort_values(by='Jaccard Sim', ascending =False).head(10)

,Word,Jaccard Sim,Shared Set,Gold Set,Silver Set
3679,facio,0.250000,"{ille, dignus, dico, quaero, miser, malum, si,...","(cura, credo, bonus, si, possum, dico, tu, ist...","(credo, non, malum, scio, iste, homo, qui, hon..."
2856,dico,0.204819,"{ille, hic, dignus, non, quaero, inquam, facio...","(facio, tu, iste, credo, ego, causa, si, bonus...","(inquam, scio, possum, tu, credo, ego, satis, ..."
7341,os,0.190476,"{simul, mons, puer, ager, redeo, oculus, fugio...","(ingens, ora, simul, ager, ipse, ferrum, membr...","(sanguis, mater, ager, mare, mons, ubi, simul,..."
1646,centum,0.190476,"{simul, puer, capio, suus, ager, redeo, fugio,...","(ingens, lego, ager, cado, puer, ora, anima, l...","(donec, mater, altus, traho, ager, regnum, qua..."
2351,credo,0.176471,"{ille, dignus, quaero, inquam, qui, paruus, op...","(bonus, animus, si, nihil, cura, mens, tantus,...","(scio, sapiens, honestus, ratio, opus, par, en..."
4724,ignis,0.176471,"{mare, mons, undus, ager, fugio, ferrum, sol, ...","(flamma, ara, litor, flumen, aqua, undus, auro...","(iaceo, sanguis, aqua, bibo, ager, pes, mare, ..."
1575,causa,0.176471,"{ille, hic, dignus, quaero, malum, inquam, opu...","(tu, si, nullus, iste, hic, credo, mens, poena...","(honestus, opus, credo, difficilis, ratio, par..."
1901,colla,0.162791,"{mons, dens, auro, undus, sanguis, ager, os, o...","(auris, aqua, litor, lego, flamma, equus, ager...","(acies, flumen, mons, simul, pes, dens, niger,..."
6884,nihil,0.162791,"{ille, dignus, non, quaero, amicus, opus, quis...","(si, opus, bonus, credo, paruus, alius, nosco,...","(scio, non, credo, tu, qui, inquam, possum, di..."
8785,ras,0.162791,"{simul, puer, mons, clamor, ager, ferrum, pes,...","(ingens, cado, in, aqua, traho, urbis, siue, s...","(nobilis, addo, rus, mollis, gaudeo, altus, ni..."


In [ ]:
pd.set_option('display.max_colwidth', None)
df.loc[df['Word']=='dico']

,Word,Jaccard Sim,Shared Set,Gold Set,Silver Set
2856,dico,0.204819,"{ille, hic, dignus, non, quaero, inquam, facio, miser, si, tu, modus, loquor, qui, talis, nihil, ego, credo}","(facio, tu, iste, credo, ego, causa, si, bonus, qui, amor, poena, animus, quaero, nullus, nihil, non, noster, refero, cura, tuus, opus, nosco, magnus, reddo, ille, res, se, modus, munus, ullus, dignus, fatum, miser, inquam, hic, audio, do, ago, tantus, loquor, licet, alius, mens, fortuna, vos, aio, talis, carmen, habeo, cognosco)","(inquam, scio, possum, tu, credo, ego, satis, beatus, ille, debeo, sapiens, modus, puto, par, necesse, ratio, nihil, irascor, ira, non, enim, quaero, patior, dignus, amicus, miser, turpis, verum, animal, contineo, qui, si, vis, hic, quis, stultus, talis, quod, ut, facio, nemo, timeo, sententia, loquor, honestus, difficilis, nego, itaque, bene, iniurius)"


In [ ]:
df.loc[df['Word']=='candidus']['Gold Set']

1412    (armis, uns, ingens, les, ipse, liber, mollis, sto, paro, urbis, puer, pes, aer, oculus, vidit, lego, ara, caput, ars, ager, cado, venit, herba, sax, mitto, amnis, summus, flumen, annus, orbis, soror, equus, dux, flamma, accipio, patior, auras, ferrum, mare, sacer, traho, aqua, qualis, lumen, fugio, suus, redeo, relinquo, siue, cursus)
Name: Gold Set, dtype: object

## Gold to Patr

In [ ]:
gsp_info = []

for x in gold_voc: 
  if x in silver_voc: 
    if x in pat_voc:
      w1 = gol_model.wv.most_similar(x, topn=50)
      w2 = pat_model.wv.most_similar(x, topn=50)
      sim = j_sim(w1,w2)
      topgold = list(zip(*w1))[0] 
      toppat = list(zip(*w2))[0]
      intersect = set(topgold).intersection(set(toppat))
      gsp_info.append((x,sim,intersect,topgold,toppat))

In [ ]:
data = list(sorted(gsp_info))
df2 = pd.DataFrame(data, columns =['Word','Jaccard Sim',"Shared Set",'Gold Set','Pat Set'])

In [ ]:
df2.sort_values(by='Jaccard Sim', ascending =False)

,Word,Jaccard Sim,Shared Set,Gold Set,Pat Set
4095,quod,0.204819,"{ille, hic, res, et, non, tamen, quaero, ut, q...","(non, ego, cura, quis, tu, opus, et, poena, po...","(ut, se, ego, qui, is, et, cum, enim, tu, aute..."
4808,sum,0.204819,"{ille, omnis, ego, et, non, bonus, quod, qui, ...","(tuus, ego, tu, facio, nullus, amor, ., causa,...","(is, non, ipse, et, ego, jam, ille, qui, tu, h..."
3047,modus,0.190476,"{ille, res, et, non, liber, cum, quis, vos, si...","(et, si, malum, amor, amicus, ille, licet, res...","(res, christianus, ab, debeo, quidam, qui, loc..."
3768,possum,0.190476,"{ille, hic, ipse, animus, et, non, tamen, qui,...","(non, si, cura, amicus, qui, et, carmen, mens,...","(ille, is, ut, se, et, ego, vos, multus, qui, ..."
1485,ego,0.176471,"{ille, res, et, non, tamen, ut, qui, quis, si,...","(tu, possum, si, qui, non, amicus, opus, amor,...","(tu, quod, hic, qui, et, is, ille, ut, cum, ta..."
...,...,...,...,...,...
3835,praesumo,0.000000,{},"(conclamo, veniente, cereus, probitas, lucina,...","(habeo, nec, iste, locus, nunc, scribo, quanto..."
3836,praetempto,0.000000,{},"(agricola, decles, orbus, acetus, fragilis, pu...","(lugeo, conversatione, aptus, veluti, signific..."
3837,praetendo,0.000000,{},"(iuvenum, ratio, ora, odi, pecus, odium, iuven...","(judicanti, nubilis, castellum, vocasti, 11, c..."
1643,explorator,0.000000,{},"(catullum, pertimuo, sergestus, percudro, favo...","(cottidie, dono, nequaquam, purgo, magnitudo, ..."


In [ ]:
df2.loc[df2['Word']=='candidus']#[['Word','Jaccard Sim','Shared Set']]

,Word,Jaccard Sim,Shared Set,Gold Set,Pat Set
599,candidus,0.030928,"{mos, annus, templum}","(paruus, dux, mater, ager, annus, cum, malum, ...","(templum, periculum, similis, fons, multo, fra..."


In [ ]:
df2.loc[df2['Word']=='albus']#[['Word','Jaccard Sim','Shared Set']]

,Word,Jaccard Sim,Shared Set,Gold Set,Pat Set
211,albus,0.052632,"{puer, morior, suus, cum, gens}","(caput, ipse, redeo, lego, relinquo, ager, durus, oculus, populus, pono, inque, equus, paruus, amicus, aio, puer, soleo, mollis, liber, cum, honor, malum, gens, ars, dexter, aqua, aer, fugio, rex, lingua, siue, dux, accipio, membrum, morior, pes, suus, ora, audio, ingens, duo, flumen, iam, auro, sumo, sacer, iubeo, armis, duco, mos)","(contineo, contra, nunc, puer, unde, nihil, de, multum, morior, natura, vita, potius, misericordia, sine, tam, sensus, damno, nomen, cum, utrum, scio, tunc, in, suus, itaque, animus, gens, pater, vitae, quidam, vero, solum, quanto, imago, tamquam, inde, tuus, usque, potestas, ut, materia, noster, patior, tantus, desum, prohibeo, quod, sed, ego, plane)"


##Silver to Pat

In [ ]:
sp_info = []

for x in silver_voc: 
    if x in pat_voc:
      w1 = sil_model.wv.most_similar(x, topn=50)
      w2 = pat_model.wv.most_similar(x, topn=50)
      sim = j_sim(w1,w2)
      topsil = list(zip(*w1))[0] 
      toppat = list(zip(*w2))[0]
      intersect = set(topsil).intersection(set(toppat))
      sp_info.append((x,sim,intersect,topsil,toppat))

In [ ]:
data = list(sorted(sp_info))
df3 = pd.DataFrame(data, columns =['Word','Jaccard Sim',"Shared Set",'Silver Set','Pat Set'])

In [ ]:
df3.sort_values(by='Jaccard Sim', ascending =False).head(1)

,Word,Jaccard Sim,Shared Set,Silver Set,Pat Set
1879,dico,0.234568,"{ille, debeo, quaero, ut, puto, scio, enim, si, tu, talis, hic, ego, credo, non, quis, qui, nihil, loquor, possum}","(inquam, scio, possum, tu, credo, ego, satis, beatus, ille, debeo, sapiens, modus, puto, par, necesse, ratio, nihil, irascor, ira, non, enim, quaero, patior, dignus, amicus, miser, turpis, verum, animal, contineo, qui, si, vis, hic, quis, stultus, talis, quod, ut, facio, nemo, timeo, sententia, loquor, honestus, difficilis, nego, itaque, bene, iniurius)","(ego, se, hic, et, si, is, ille, ipse, enim, jam, sicut, credo, cum, quaero, nihil, puto, autem, qui, loquor, iste, ibi, tu, intellego, possum, audio, in, non, alius, quis, nescio, ut, nunc, sum, debeo, quia, habeo, scio, talis, ad, tunc, nos, causa, tamen, multus, nolo, bonus, quidam, ab, homo, quando)"


In [ ]:
df3.loc[df3['Word']=='dico']#[['Word','Jaccard Sim','Shared Set']]

,Word,Jaccard Sim,Shared Set,Silver Set,Pat Set
1879,dico,0.234568,"{ille, debeo, quaero, ut, puto, scio, enim, si, tu, talis, hic, ego, credo, non, quis, qui, nihil, loquor, possum}","(inquam, scio, possum, tu, credo, ego, satis, beatus, ille, debeo, sapiens, modus, puto, par, necesse, ratio, nihil, irascor, ira, non, enim, quaero, patior, dignus, amicus, miser, turpis, verum, animal, contineo, qui, si, vis, hic, quis, stultus, talis, quod, ut, facio, nemo, timeo, sententia, loquor, honestus, difficilis, nego, itaque, bene, iniurius)","(ego, se, hic, et, si, is, ille, ipse, enim, jam, sicut, credo, cum, quaero, nihil, puto, autem, qui, loquor, iste, ibi, tu, intellego, possum, audio, in, non, alius, quis, nescio, ut, nunc, sum, debeo, quia, habeo, scio, talis, ad, tunc, nos, causa, tamen, multus, nolo, bonus, quidam, ab, homo, quando)"
